# Big Data Analytics of Hotel Bookings

### Data Import

In [1]:
# import modules
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
# build spark session and spark context
spark = SparkSession.builder \
        .appName("hotel") \
        .getOrCreate()
sc = spark.sparkContext

In [3]:
df = spark.read.csv('hotel_bookings.csv',  inferSchema=True, header = True)
df.take(2)

[Row(hotel='Resort Hotel', is_canceled=0, lead_time=342, arrival_date_year=2015, arrival_date_month='July', arrival_date_week_number=27, arrival_date_day_of_month=1, stays_in_weekend_nights=0, stays_in_week_nights=0, adults=2, children='0', babies=0, meal='BB', country='PRT', market_segment='Direct', distribution_channel='Direct', is_repeated_guest=0, previous_cancellations=0, previous_bookings_not_canceled=0, reserved_room_type='C', assigned_room_type='C', booking_changes=3, deposit_type='No Deposit', agent='NULL', company='NULL', days_in_waiting_list=0, customer_type='Transient', adr=0.0, required_car_parking_spaces=0, total_of_special_requests=0, reservation_status='Check-Out', reservation_status_date='2015-07-01'),
 Row(hotel='Resort Hotel', is_canceled=0, lead_time=737, arrival_date_year=2015, arrival_date_month='July', arrival_date_week_number=27, arrival_date_day_of_month=1, stays_in_weekend_nights=0, stays_in_week_nights=0, adults=2, children='0', babies=0, meal='BB', country='

In [4]:
# view columns and schema
df.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

### Data Preprocessing

In [5]:
# replace the strings "NULL" and "NA" with null value
df_withNull = df.replace('NULL', None).replace('NA', None)

# view number of null values per column
from pyspark.sql.functions import isnan, when, count, col
df_withNull.select([count(when(col(c).isNull(), c)).alias(c) for c in df_withNull.columns]).show()

+-----+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+-------------+---+---------------------------+-------------------------+------------------+-----------------------+
|hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|days_in_waiting_list|customer_type|adr|required_car_parking_spaces|total_of_special_reque

In [6]:
# replace null values in 'children' to 0 since there are only 4
df2 = df_withNull.fillna({'children':0})

# replace 'children' datatype to int
df2 = df2.withColumn('children', col('children').cast("Int"))

In [7]:
# drop 'company' and 'agent' due to high null count
df2 = df2.drop('agent', 'company','country', 'arrival_date_week_number')



In [8]:
df2.dtypes

[('hotel', 'string'),
 ('is_canceled', 'int'),
 ('lead_time', 'int'),
 ('arrival_date_year', 'int'),
 ('arrival_date_month', 'string'),
 ('arrival_date_day_of_month', 'int'),
 ('stays_in_weekend_nights', 'int'),
 ('stays_in_week_nights', 'int'),
 ('adults', 'int'),
 ('children', 'int'),
 ('babies', 'int'),
 ('meal', 'string'),
 ('market_segment', 'string'),
 ('distribution_channel', 'string'),
 ('is_repeated_guest', 'int'),
 ('previous_cancellations', 'int'),
 ('previous_bookings_not_canceled', 'int'),
 ('reserved_room_type', 'string'),
 ('assigned_room_type', 'string'),
 ('booking_changes', 'int'),
 ('deposit_type', 'string'),
 ('days_in_waiting_list', 'int'),
 ('customer_type', 'string'),
 ('adr', 'double'),
 ('required_car_parking_spaces', 'int'),
 ('total_of_special_requests', 'int'),
 ('reservation_status', 'string'),
 ('reservation_status_date', 'string')]

In [9]:
df2.groupBy('arrival_date_month').count().show()

+------------------+-----+
|arrival_date_month|count|
+------------------+-----+
|              July|12661|
|          November| 6794|
|          February| 8068|
|           January| 5929|
|             March| 9794|
|           October|11160|
|               May|11791|
|            August|13877|
|             April|11089|
|              June|10939|
|          December| 6780|
|         September|10508|
+------------------+-----+



In [10]:
#month_num = df2.replace('January', '1')\
#                .replace('February', '2')\
#                .replace('March', '3')\
#                .replace('April', '4')\
#                .replace('May', '5')\
#                .replace('June', '6')



In [11]:
#month_num = month_num.replace('July', '7')\
#                      .replace('August', '8')\
#                .replace('September', '9')\
#                .replace('October', '10')\
#                .replace('November', '11')\
#                .replace('December', '12')



**Data Preprocessing Todo's**
- map arrival_date_month from strings to numbers: so "January" = 1, etc.
- numerically encode all the other string variables, otherwise remove them  

In [12]:
# numerically encode arrival_date_month


...

Ellipsis

In [13]:
# for 'meal': undefined/SC are same, they mean no meal package. BB is bed&breakfast, HB is breakfast and dinner, and FB is full breakfast lunch and dinner

# replace Undefined with SC in meal package
df2 = df2.replace('Undefined', 'SC')

# see value counts for 'meal'
df2.groupBy('meal').count().show()

+----+-----+
|meal|count|
+----+-----+
|  SC|11819|
|  FB|  798|
|  BB|92310|
|  HB|14463|
+----+-----+



In [14]:
#numerically encode all columns of type string

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderModel, StringIndexer


col_string=['hotel', 'arrival_date_month','meal','market_segment','distribution_channel','reserved_room_type','assigned_room_type','deposit_type','customer_type','reservation_status','reservation_status_date']
col_num=[x+"_NUMERIC" for x in col_string]

#add other columns for encoding
col_num=col_num+['arrival_date_year','arrival_date_day_of_month']

col_oh=[x+"_oh" for x in col_string]

col_oh=col_oh+['arrival_date_year_oh','arrival_date_day_of_month_oh']



In [15]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_NUMERIC").fit(df2) for column in col_string]
pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(df2).transform(df2)



In [16]:

df_indexed.dtypes

[('hotel', 'string'),
 ('is_canceled', 'int'),
 ('lead_time', 'int'),
 ('arrival_date_year', 'int'),
 ('arrival_date_month', 'string'),
 ('arrival_date_day_of_month', 'int'),
 ('stays_in_weekend_nights', 'int'),
 ('stays_in_week_nights', 'int'),
 ('adults', 'int'),
 ('children', 'int'),
 ('babies', 'int'),
 ('meal', 'string'),
 ('market_segment', 'string'),
 ('distribution_channel', 'string'),
 ('is_repeated_guest', 'int'),
 ('previous_cancellations', 'int'),
 ('previous_bookings_not_canceled', 'int'),
 ('reserved_room_type', 'string'),
 ('assigned_room_type', 'string'),
 ('booking_changes', 'int'),
 ('deposit_type', 'string'),
 ('days_in_waiting_list', 'int'),
 ('customer_type', 'string'),
 ('adr', 'double'),
 ('required_car_parking_spaces', 'int'),
 ('total_of_special_requests', 'int'),
 ('reservation_status', 'string'),
 ('reservation_status_date', 'string'),
 ('hotel_NUMERIC', 'double'),
 ('arrival_date_month_NUMERIC', 'double'),
 ('meal_NUMERIC', 'double'),
 ('market_segment_NUMER

In [17]:
#one-hot encode all columns in col_num

ohe = OneHotEncoder(dropLast=False)
ohe.setInputCols(col_num)
ohe.setOutputCols(col_oh)
model = ohe.fit(df_indexed)

df_casted=model.transform(df_indexed)


In [18]:
df_casted.dtypes

[('hotel', 'string'),
 ('is_canceled', 'int'),
 ('lead_time', 'int'),
 ('arrival_date_year', 'int'),
 ('arrival_date_month', 'string'),
 ('arrival_date_day_of_month', 'int'),
 ('stays_in_weekend_nights', 'int'),
 ('stays_in_week_nights', 'int'),
 ('adults', 'int'),
 ('children', 'int'),
 ('babies', 'int'),
 ('meal', 'string'),
 ('market_segment', 'string'),
 ('distribution_channel', 'string'),
 ('is_repeated_guest', 'int'),
 ('previous_cancellations', 'int'),
 ('previous_bookings_not_canceled', 'int'),
 ('reserved_room_type', 'string'),
 ('assigned_room_type', 'string'),
 ('booking_changes', 'int'),
 ('deposit_type', 'string'),
 ('days_in_waiting_list', 'int'),
 ('customer_type', 'string'),
 ('adr', 'double'),
 ('required_car_parking_spaces', 'int'),
 ('total_of_special_requests', 'int'),
 ('reservation_status', 'string'),
 ('reservation_status_date', 'string'),
 ('hotel_NUMERIC', 'double'),
 ('arrival_date_month_NUMERIC', 'double'),
 ('meal_NUMERIC', 'double'),
 ('market_segment_NUMER

In [19]:
#check oh encoded df_casted


df_casted.groupBy('booking_changes').count().show()




+---------------+------+
|booking_changes| count|
+---------------+------+
|             12|     2|
|              1| 12701|
|             13|     5|
|              6|    63|
|             16|     2|
|              3|   927|
|             20|     1|
|              5|   118|
|             15|     3|
|             17|     2|
|              9|     8|
|              4|   376|
|              8|    17|
|              7|    31|
|             10|     6|
|             21|     1|
|             11|     2|
|             14|     5|
|              2|  3805|
|              0|101314|
+---------------+------+
only showing top 20 rows



In [20]:
df_casted.groupBy('arrival_date_year_oh').count().show()

+--------------------+-----+
|arrival_date_year_oh|count|
+--------------------+-----+
| (2018,[2017],[1.0])|40687|
| (2018,[2015],[1.0])|21996|
| (2018,[2016],[1.0])|56707|
+--------------------+-----+



In [30]:
df_encoded=df_casted.drop(*col_string)
df_encoded=df_encoded.drop(*col_num)

In [22]:
df_encoded.dtypes

[('is_canceled', 'int'),
 ('lead_time', 'int'),
 ('stays_in_weekend_nights', 'int'),
 ('stays_in_week_nights', 'int'),
 ('adults', 'int'),
 ('children', 'int'),
 ('babies', 'int'),
 ('is_repeated_guest', 'int'),
 ('previous_cancellations', 'int'),
 ('previous_bookings_not_canceled', 'int'),
 ('booking_changes', 'int'),
 ('days_in_waiting_list', 'int'),
 ('adr', 'double'),
 ('required_car_parking_spaces', 'int'),
 ('total_of_special_requests', 'int'),
 ('distribution_channel_oh', 'vector'),
 ('customer_type_oh', 'vector'),
 ('reservation_status_date_oh', 'vector'),
 ('arrival_date_day_of_month_oh', 'vector'),
 ('reservation_status_oh', 'vector'),
 ('market_segment_oh', 'vector'),
 ('reserved_room_type_oh', 'vector'),
 ('assigned_room_type_oh', 'vector'),
 ('meal_oh', 'vector'),
 ('hotel_oh', 'vector'),
 ('deposit_type_oh', 'vector'),
 ('arrival_date_year_oh', 'vector'),
 ('arrival_date_month_oh', 'vector')]

In [23]:
df_encoded.groupBy('arrival_date_month_oh').count().show()

+---------------------+-----+
|arrival_date_month_oh|count|
+---------------------+-----+
|       (12,[0],[1.0])|13877|
|       (12,[9],[1.0])| 6794|
|       (12,[8],[1.0])| 8068|
|      (12,[10],[1.0])| 6780|
|       (12,[1],[1.0])|12661|
|       (12,[4],[1.0])|11089|
|       (12,[3],[1.0])|11160|
|       (12,[6],[1.0])|10508|
|       (12,[5],[1.0])|10939|
|       (12,[2],[1.0])|11791|
|       (12,[7],[1.0])| 9794|
|      (12,[11],[1.0])| 5929|
+---------------------+-----+



In [24]:
df_encoded.show(2)

+-----------+---------+-----------------------+--------------------+------+--------+------+-----------------+----------------------+------------------------------+---------------+--------------------+---+---------------------------+-------------------------+-----------------------+----------------+--------------------------+----------------------------+---------------------+-----------------+---------------------+---------------------+-------------+-------------+---------------+--------------------+---------------------+
|is_canceled|lead_time|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|booking_changes|days_in_waiting_list|adr|required_car_parking_spaces|total_of_special_requests|distribution_channel_oh|customer_type_oh|reservation_status_date_oh|arrival_date_day_of_month_oh|reservation_status_oh|market_segment_oh|reserved_room_type_oh|assigned_room_type_oh|      meal_oh|     hotel_oh|deposi

In [25]:
columns_ = df_encoded.columns
columns_

['is_canceled',
 'lead_time',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'distribution_channel_oh',
 'customer_type_oh',
 'reservation_status_date_oh',
 'arrival_date_day_of_month_oh',
 'reservation_status_oh',
 'market_segment_oh',
 'reserved_room_type_oh',
 'assigned_room_type_oh',
 'meal_oh',
 'hotel_oh',
 'deposit_type_oh',
 'arrival_date_year_oh',
 'arrival_date_month_oh']

In [44]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

df_encoded=df_casted.drop(*col_string)
df_encoded=df_encoded.drop(*col_num)

#columns of all vector type data that are going to be normalied 
columns_ = ['distribution_channel_oh',
 'customer_type_oh',
 'reservation_status_date_oh',
 'arrival_date_day_of_month_oh',
 'reservation_status_oh',
 'market_segment_oh',
 'reserved_room_type_oh',
 'assigned_room_type_oh',
 'meal_oh',
 'hotel_oh',
 'deposit_type_oh',
 'arrival_date_year_oh',
 'arrival_date_month_oh']


#normalizing all vector type data with l^1 norm
for column in columns_:
    normalizer = Normalizer(inputCol=column, outputCol=column+"_NORM", p=1.0) 
    l1NormData = normalizer.transform(df_encoded)
    df_encoded = l1NormData
l1NormData = df_encoded
print("Normalized using L^1 norm")

l1NormData = l1NormData.drop(*columns_)
#l1NormData.show()
    



Normalized using L^1 norm


In [45]:
l1NormData.dtypes

[('is_canceled', 'int'),
 ('lead_time', 'int'),
 ('stays_in_weekend_nights', 'int'),
 ('stays_in_week_nights', 'int'),
 ('adults', 'int'),
 ('children', 'int'),
 ('babies', 'int'),
 ('is_repeated_guest', 'int'),
 ('previous_cancellations', 'int'),
 ('previous_bookings_not_canceled', 'int'),
 ('booking_changes', 'int'),
 ('days_in_waiting_list', 'int'),
 ('adr', 'double'),
 ('required_car_parking_spaces', 'int'),
 ('total_of_special_requests', 'int'),
 ('distribution_channel_oh_NORM', 'vector'),
 ('customer_type_oh_NORM', 'vector'),
 ('reservation_status_date_oh_NORM', 'vector'),
 ('arrival_date_day_of_month_oh_NORM', 'vector'),
 ('reservation_status_oh_NORM', 'vector'),
 ('market_segment_oh_NORM', 'vector'),
 ('reserved_room_type_oh_NORM', 'vector'),
 ('assigned_room_type_oh_NORM', 'vector'),
 ('meal_oh_NORM', 'vector'),
 ('hotel_oh_NORM', 'vector'),
 ('deposit_type_oh_NORM', 'vector'),
 ('arrival_date_year_oh_NORM', 'vector'),
 ('arrival_date_month_oh_NORM', 'vector')]

In [46]:
# convert to RDD
# data_rdd = df2.rdd

data_rdd = df_encoded.rdd

# 70/30 test train split
train, test = data_rdd.randomSplit([0.7, 0.3], seed=42)



### Exploratory Data Analysis

helpful links:
- https://spark.apache.org/docs/latest/ml-guide.html 
